현재는 입차 출차 코드 모두 왼쪽에서 > 오른쪽으로 기준선을 넘어갈 때 최초 1회만 인식하도록 만든 상태

오른쪽에서 왼쪽으로 넘어가는걸 카운트하고싶은경우

if prev_cx is not None and prev_cx >= line_x and cx < line_x and track_id not in counted_ids: 이부분을

> > if prev_cx is not None and prev_cx <= line_x and cx > line_x and track_id not in counted_ids: 이렇게 변경하면됨


In [ ]:
!pip install sort-tracker

In [ ]:
!pip install filterpy

In [ ]:
import pyodbc

# 연결 문자열
conn_str = (
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=databasefor-bi.database.windows.net;"
    "DATABASE=databasebi;"
    "UID=CloudSA6a9bfa5a;"
    "PWD=6aiteam10!;"
    "Encrypt=yes;"
    "TrustServerCertificate=no;"
)

# 연결 테스트
try:
    print("⏳ SQL Server에 연결 중...")
    conn = pyodbc.connect(conn_str, timeout=10)
    print("✅ 연결 성공!")
    conn.close()
except Exception as e:
    print("❌ 연결 실패:", e)



In [ ]:
import time  # ← 추가
from ultralytics import YOLO
import cv2
import numpy as np
import requests
import os
import datetime
import pyodbc
from uuid import uuid4
from sort import Sort
from datetime import datetime
from datetime import datetime, timedelta
import pytz
from azure.storage.blob import BlobServiceClient
from datetime import datetime


def upload_image_to_blob(image, plate_text):
    try:
        # 현재 시간 기준 파일 이름 생성
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"{plate_text}_{timestamp}.jpg"

        # 이미지 임시 저장
        temp_path = f"temp_upload_{uuid4().hex}.jpg"
        cv2.imwrite(temp_path, image)

        # Blob 서비스 클라이언트 생성
        blob_service_client = BlobServiceClient.from_connection_string(AZURE_CONNECTION_STRING)
        blob_client = blob_service_client.get_blob_client(container=CONTAINER_NAME, blob=filename)

        # 업로드
        with open(temp_path, "rb") as data:
            blob_client.upload_blob(data)

        os.remove(temp_path)
        print(f"✅ Azure Blob에 업로드 완료: {filename}")
        return blob_client.url  # 저장된 URL 반환
    except Exception as e:
        print("❌ 이미지 업로드 실패:", e)
        return None

def get_kst_now():
    return datetime.now(pytz.timezone('Asia/Seoul'))

def save_entry_to_db(plate):
    conn = None
    try:
        conn = pyodbc.connect(conn_str)
        cursor = conn.cursor()
        cursor.execute("EXEC sp_VehicleEntry @차량번호 = ?", plate)
        conn.commit()
        print("✅ 입차 저장 완료")
    except Exception as e:
        print("❌ 입차 저장 오류:", e)
    finally:
        if conn:
            conn.close()


def get_plate_text_from_ocr_api(cropped_img):
    temp_filename = f"temp_{uuid4().hex}.jpg"
    cv2.imwrite(temp_filename, cropped_img)
    try:
        with open(temp_filename, "rb") as f:
            files = {"file": (temp_filename, f, "image/jpeg")}
            response = requests.post(OCR_ENDPOINT, files=files)
        if response.status_code == 200:
            return response.json().get("ocr_results", [""])[0]
        else:
            print("❌ OCR 응답 실패:", response.status_code)
            return None
    except Exception as e:
        print("❌ OCR 요청 중 오류:", e)
        return None
    finally:
        os.remove(temp_filename)

video_path = "newenw각도1.mp4"
cap = cv2.VideoCapture(video_path)

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
line_x = 1600

entry_count = 0
counted_ids = set()
previous_edges = {}

# ✅ 시간 간격 제어 변수 추가
last_processed_time = 0
process_interval = 1.0  # 1초마다 처리

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    current_time = time.time()
    if current_time - last_processed_time < process_interval:
        continue  # 1초 안 지났으면 건너뜀
    last_processed_time = current_time  # 타이머 갱신

    results = model.predict(source=frame, classes=[2, 5, 7], conf=0.4)
    detections = []

    for box in results[0].boxes:
        x1, y1, x2, y2, conf = map(float, box.xyxy[0].tolist() + [box.conf[0]])
        detections.append([x1, y1, x2, y2, conf])

    dets = np.array(detections)
    tracks = tracker.update(dets) if dets.shape[0] > 0 else []

    for track in tracks:
        x1, y1, x2, y2, track_id = map(int, track)
        cy = int((y1 + y2) / 2)
        prev_x2 = previous_edges.get(track_id)
        previous_edges[track_id] = x2

        if prev_x2 is not None and prev_x2 <= line_x and x2 > line_x and track_id not in counted_ids:
            counted_ids.add(track_id)
            entry_count += 1

            cropped = frame[y1:y2, x1:x2]
            plate_text = get_plate_text_from_ocr_api(cropped)
            entry_time = get_kst_now().strftime('%Y-%m-%d %H:%M:%S')  # 문자열로 변환
  # datetime 객체 그대로 사용!



            print(f"✅ 차량 {track_id} 입차 감지! 번호판: {plate_text} | 시간: {entry_time} | 총 차량 수: {entry_count}")
            save_entry_to_db(plate_text)
            upload_url = upload_image_to_blob(cropped, plate_text)

        cx = int((x1 + x2) / 2)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"ID {track_id}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1)

    cv2.line(frame, (line_x, 0), (line_x, frame.shape[0]), (255, 255, 0), 3)
    cv2.putText(frame, f"Total: {entry_count}", (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 3)

    resized_frame = cv2.resize(frame, (1280, 720))
    cv2.imshow("Vehicle Entry with SORT + OCR", resized_frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()



0: 640x384 1 car, 56.1ms
Speed: 2.1ms preprocess, 56.1ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 car, 56.3ms
Speed: 1.5ms preprocess, 56.3ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 car, 52.7ms
Speed: 1.3ms preprocess, 52.7ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 car, 56.8ms
Speed: 1.2ms preprocess, 56.8ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 car, 59.9ms
Speed: 1.4ms preprocess, 59.9ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 car, 61.0ms
Speed: 1.5ms preprocess, 61.0ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 384)
✅ 차량 10 입차 감지! 번호판: 23무0704 | 시간: 2025-06-05 18:00:34 | 총 차량 수: 1
✅ 입차 저장 완료
✅ Azure Blob에 업로드 완료: 23무0704_20250605_180034.jpg

0: 640x384 1 car, 66.1ms
Speed: 2.1ms preprocess, 66.1ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 